# Accuinsight+ Sandbox 시작하기: Scikit-learn, DecisionTreeClassifier

목차  

1. [데이터 가져오기](#데이터-가져오기)
2. [모델 작성](#모델-작성)
3. [데이터 준비](#데이터-준비)
4. [모델 훈련](#모델-훈련)
5. [모델 배포 테스트](#모델-배포-테스트)

이 문서는 Accuinsight+ 3.0 Sandbox의 Scikit-learn 노트북 예제 파일입니다.  
본 예제에서는 iris tabular 데이터를 이용하여 DecisionTreeClassifier 모델을 훈련하고 Accuinsight+ 3.0에 배포 할 준비를 해보겠습니다. 

## 데이터 가져오기

본 문서에서는 iris 데이터셋 노드가 샌드박스 노드에 연결되어 있음을 전제로 합니다.  
iris 데이터셋 노드가 연결되어 있지 않다면 별도의 작업을 통해 데이터셋을 다운로드 받아서 사용 가능합니다.  

만약 iris가 아닌 다른 데이터셋 노드를 연결하여 사용하고 싶다면 예제를 실행하며 데이터에 알맞게 코드를 변형하여야 합니다.   

아래 코드를 실행하면 샌드박스 노드에 연결된 데이터셋 노드의 데이터를 가져옵니다.  
만약 2개 이상의 노드가 연결되어 있다면 첫 번째로 연결한 노드의 데이터를 가져옵니다. 

In [5]:
%run -i /home/work/airuntime/startup.py

Filename: /home/work/dataset/iris.csv
Dataset has been loaded as 'dataset'.

'dataset' type: <class 'pandas.core.frame.DataFrame'>

             non_null/total counts    dtype  dtype_from
sepal_length               150/150  float64  DoubleType
sepal_width                150/150  float64  DoubleType
petal_length               150/150  float64  DoubleType
petal_width                150/150  float64  DoubleType
variety                    150/150   object  StringType

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Columns: 5 entries, sepal_length to variety
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


---
## 모델 작성
아래 세 가지의 파일을 통해 사용할 모델을 직접 작성할 수 있습니다.     
아래 `model.py`, `preprocessor.py`, `postprocessor.py` 링크를 클릭해서 해당 파일들을 편집할 수 있습니다.   
단, 작성중인 ipynb 파일의 위치가 `/home/work` 가 아닌 경우에는 이 셀을 편집하여 파일의 링크를 `/home/work` 에서 접근 가능한 상대 경로로 수정하여야 합니다.   
예를 들어, 만약 첫 화면에서 `my_notebooks` 라는 폴더를 생성하고 해당 위치에서 ipynb 파일을 작성한다면 `model.py` 의 링크를 `../` 로 시작하도록 수정합니다.   

* [`model.py`](./airuntime/sklearn/sklearnserver/trainer/model.py) 실제 사용되는 모델 파일입니다. 아래 build, save, load 세 함수를 편집하면 됩니다. 
  * `build`
  * `save`
  * `load`
* [`preprocessor.py`](./airuntime/sklearn/sklearnserver/trainer/preprocessor.py) 전처리 코드입니다. Workflow에서 전처리 노드를 작업한 경우 해당 코드가 들어있습니다. 
* [`postprocessor.py`](./airuntime/sklearn/sklearnserver/trainer/postprocessor.py) 후처리 코드입니다. 모델의 예측 결과를 후처리하는 코드를 작성합니다. 

---
## 데이터 준비 

모델에 사용 할 데이터를 준비합니다.  
샌드박스 노드에 데이터셋 노드를 연결해두었다면 앞서 '데이터 가져오기' 코드에서 dataset 객체가 생성된 상태입니다.   
dataset은 Pandas DataFrame 객체입니다.

In [6]:
print(dataset.head())

   sepal_length  sepal_width  petal_length  petal_width variety
0           5.1          3.5           1.4          0.2  Setosa
1           4.9          3.0           1.4          0.2  Setosa
2           4.7          3.2           1.3          0.2  Setosa
3           4.6          3.1           1.5          0.2  Setosa
4           5.0          3.6           1.4          0.2  Setosa


읽어들인 dataset의 정보를 확인해보겠습니다.

In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   variety       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


Target column을 명시해줍니다.  
현재 iris 데이터를 기준으로 예제가 작성되어 있으므로, 만약 다른 데이터를 사용하고 있다면 그에 맞게 Target column을 지정합니다.  

예제 iris 데이터의 훈련 대상 column은 SepalLength, SepalWidth, PetalLength, PetalWidth 이고  
Target column은 variety 입니다. 

In [9]:
_TARGET_COL_ = col_y = "variety"

dataset = dataset.dropna()
data_x = dataset[dataset.columns.drop([col_y])].astype("float")

In [10]:
print(data_x.head())

   sepal_length  sepal_width  petal_length  petal_width
0           5.1          3.5           1.4          0.2
1           4.9          3.0           1.4          0.2
2           4.7          3.2           1.3          0.2
3           4.6          3.1           1.5          0.2
4           5.0          3.6           1.4          0.2


---
## 모델 훈련

앞서 준비한 데이터로 모델을 훈련합니다. 

In [11]:
%load_ext autoreload
%autoreload 2

from airuntime.sklearn import sklearnserver as modelserver

/opt/conda/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


### 모듈을 다시 로드

In [12]:
import importlib
importlib.reload(modelserver)

<module 'airuntime.sklearn.sklearnserver' from '/home/work/airuntime/sklearn/sklearnserver/__init__.py'>

모델을 빌드합니다.  
별도로 편집하지 않은 경우 예제로 사용하는 DecisionTreeClassifier 모델을 그대로 빌드합니다. 

In [13]:
model = modelserver.trainer.model.SKLearnModel()
model.build()

### 훈련

작성한 모델과 데이터를 이용해 훈련을 시작합니다.

In [14]:
model.train(data_x)

### 평가

모델을 평가합니다

In [15]:
model.evaluate(data_x)

0.5528190123564102

### 모델 예측 테스트

모델 훈련 후 예측이 가능한지 테스트합니다.

In [16]:
model.predict(data_x)[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

### 모델 저장

모델을 배포하기위해 저장합니다. 

In [17]:
model.save("/home/work/airuntime/sklearn/models")

### 모델 불러오기 테스트

배포 전에 모델 저장이 잘 수행되었는 지 확인합니다

In [18]:
loaded_model = modelserver.trainer.model.SKLearnModel(dirpath="/home/work/airuntime/sklearn/models")

### 저장한 모델 예측 테스트

예측결과가 저장한 후 모델의 예측 결과와 동일한 지 확인합니다

In [19]:
np.array_equal(model.predict(data_x), loaded_model.predict(data_x))

True

---
## 모델 배포 테스트

모델을 정상적으로 배포가능한 지 테스트용 서버를 실행하여 API 테스트를 수행하겠습니다.

### 테스트용 서버 실행

테스트용 서버 ui를 실행합니다.  
아래 코드를 실행한 뒤, "SELECT" 버튼을 눌러서 모델 생성에 사용된 데이터 파일을 지정합니다.  
그리고 Target Col 에 Target이 될 컬럼명   
variety  
를 입력한 뒤 Test Server: Start를 클릭합니다

In [20]:
%run -i /home/work/airuntime/_widgets/run_server_ui.py -k sklearn

[I 230516 13:23:16 services:1727] object_store_memory is not verified when plasma_directory is set.


FileChooser(path='/home/work/dataset', filename='iris.csv', title='Select Dataset from <b>`dataset`</b> direct…

### 모델 예측 요청 전달

실행한 테스트용 서버에 예측 요청이 정상적으로 전달되고 결과가 나오는지 확인합니다.

In [21]:
%run -i /home/work/airuntime/_widgets/request_ui.py

#### 테스트 용 Json 요청 샘플

테스트용 서버는 Json 포맷으로 모델 예측 요청을 전달해야 합니다.   
아래 샘플들을 복사하여 상단 모델 예측 요청 전달의 json 부분에 복사하여 Send를 클릭합니다. 

* iris 모델 예측용 input data 샘플

## 예제용 데이터셋 다운로드

데이터셋이 준비되었기 때문에 바로 모델 훈련 단계로 넘어가면 됩니다.   

[모델 훈련으로 바로가기](#모델-훈련)